# Approximating the Hessian for large neural networks.

This notebook describes how to use the spectral-density package with Tensorflow2. The main entry point of this package is the `lanczos_algorithm.approximate_hessian` function, compatible with TensorFlow2 models.
  - `model`: The Keras model for which we want to compute the Hessian.
  - `dataset`: Dataset on which the model is trained. Can be a Tensorflow dataset, or more generally any iterator yielding tuples of data (X, y). If a Tensorflow dataset is used, it should be batched.
  - `order`: Rank of the approximation of the Hessian. The higher the better the approximation. See paper for more details.
  - `reduce_op`: Whether the loss function averages or sums the per sample loss. The default value is `MEAN` and should be compatible with most Keras losses, provided you didn't specify another reduction when instantiating the loss.
  - `random_seed`: Seed to use to sample the first vector in the Lanczos algorithm.

## Example 1: Full rank estimation for linear model.

We start with a simplistic usecase: we wish to train the following model:

$$ \mbox{arg}\max_\beta \sum_i (y_i - \beta^Tx_i)^2$$

As this optimization problem is quadratic, the Hessian of the loss is independent of $\beta$ and is equal to $2X^TX$. Let's verify this using `lanczos_algorithm.approximate_hessian`, and set `order` to the number of parameters to compute the exact Hessian.

We first generate some random inputs and outputs:

In [1]:
import jax
import jax.numpy as jnp
import jax.random as random
from jax import grad, jit, vmap
from flax import linen as nn
import optax
from matplotlib import pyplot as plt
import seaborn as sns
import hessian_computation
import lanczos

In [2]:
num_samples = 50
num_features = 16
key = random.PRNGKey(0)
X = random.normal(key, (num_samples, num_features))
y = random.normal(key, (num_samples,))

We then define a linear model using the Flax API:

In [3]:
class LinearModel(nn.Module):
    num_features: int

    def setup(self):
        self.dense = nn.Dense(features=1, use_bias=False)

    def __call__(self, x):
        return self.dense(x)

# Initialize the model
model = LinearModel(num_features=num_features)
params = model.init(key, X)['params']

Finally, we define a loss function that takes as input the model and a batch of examples, and return a scalar loss. Here, we simply compute the mean squared error between the predictions of the model and the true outputs:

In [4]:
def loss_fn(params, batch):
    x, y = batch
    preds = model.apply({'params': params}, x)
    return jnp.mean((y - preds) ** 2)

We then define the optimizer using Optax:

In [5]:
# Define the optimizer
optimizer = optax.adam(learning_rate=0.001)
opt_state = optimizer.init(params)

Next, we compute the Hessian using the provided JAX implementation:

In [6]:
# Compute the Hessian using the provided JAX implementation
hessian_matrix = hessian_computation.full_hessian(lambda p: loss_fn(p, (X, y)), params)

Iteration 0/16 done in 1.85s (MVP: 1.85s).
Iteration 1/16 done in 0.08s (MVP: 0.07s).
Iteration 2/16 done in 0.07s (MVP: 0.07s).
Iteration 3/16 done in 0.07s (MVP: 0.07s).
Iteration 4/16 done in 0.07s (MVP: 0.07s).
Iteration 5/16 done in 0.07s (MVP: 0.07s).
Iteration 6/16 done in 0.07s (MVP: 0.07s).
Iteration 7/16 done in 0.07s (MVP: 0.07s).
Iteration 8/16 done in 0.07s (MVP: 0.07s).
Iteration 9/16 done in 0.08s (MVP: 0.07s).
Iteration 10/16 done in 0.08s (MVP: 0.07s).
Iteration 11/16 done in 0.07s (MVP: 0.06s).
Iteration 12/16 done in 0.07s (MVP: 0.07s).
Iteration 13/16 done in 0.07s (MVP: 0.07s).
Iteration 14/16 done in 0.08s (MVP: 0.07s).
Iteration 15/16 done in 0.07s (MVP: 0.07s).


Next, we compute the approximate Hessian matrix using the Lanczos algorithm. We need to ensure that the tangent vector `v` has the same tree structure as the parameters `params`.

In [7]:
# Compute the approximate Hessian matrix using the Lanczos algorithm
def hvp_fn(v):
    v = jax.tree_util.tree_map(lambda x: x.flatten(), v)
    return hessian_computation.hvp(lambda p, b: loss_fn(p, b), params, (X, y), v)

order = num_features
tridiag, vecs = lanczos.lanczos_alg(hvp_fn, num_features, order, key)

We can check that the reconstructed Hessian is indeed equal to $2X^TX$:

In [8]:
# Verify the reconstructed Hessian
reconstructed_H = vecs.T @ tridiag @ vecs

# Plot the Hessian matrices
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.title("Hessian as estimated by Lanczos")
sns.heatmap(reconstructed_H)
plt.subplot(1, 2, 2)
plt.title("$2X^TX$")
sns.heatmap(2 * (X.T @ X))
plt.show()

## Example 2: Convnet on Cifar10

We first define a VGG16-like model (15.2M parameters) that we train a bit on Cifar10: